In [ ]:
#Query 1

In [4]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "4",
        "spark.executor.memory": "1g",
        "spark.executor.cores": "1",
        "spark.driver.memory": "2g"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4084,application_1732639283265_4024,pyspark,idle,Link,Link,None,
4087,application_1732639283265_4027,pyspark,idle,Link,Link,None,
4093,application_1732639283265_4033,pyspark,idle,Link,Link,None,
4094,application_1732639283265_4034,pyspark,idle,Link,Link,None,
4104,application_1732639283265_4044,pyspark,idle,Link,Link,None,
4105,application_1732639283265_4045,pyspark,idle,Link,Link,None,
4106,application_1732639283265_4046,pyspark,idle,Link,Link,None,
4108,application_1732639283265_4048,pyspark,idle,Link,Link,None,
4111,application_1732639283265_4051,pyspark,idle,Link,Link,None,
4112,application_1732639283265_4052,pyspark,idle,Link,Link,None,


In [5]:
#Query 1

# Spark RDD code
from pyspark.sql import SparkSession
# To log our application's execution time:
import time
import csv
from io import StringIO

sc = SparkSession \
    .builder \
    .appName("RDD query 1 execution") \
    .getOrCreate() \
    .sparkContext

start_time = time.time()

#crime data
#When using the given format the csv file was wrongly separated and it gave wrong records. That's why we created a function that is able to open the csv with respect to 
#commas inside columns.

#cd_2010_2019 = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv") \
#                   .map(lambda x: (x.split(",")))
#cd_2020_ = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv") \
#                    .map(lambda x: (x.split(",")))

def parse_csv(line):
    return next(csv.reader(StringIO(line)))

cd_2010_2019 = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv") \
    .map(parse_csv)

cd_2020_ = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv") \
    .map(parse_csv)

header_2010_2019 = cd_2010_2019.first()
header_2020 = cd_2020_.first()

data_2010_2019 = cd_2010_2019.filter(lambda x: x != header_2010_2019)
data_2020 = cd_2020_.filter(lambda x: x != header_2020)

#Filter to get only the incidents with aggravated assault
cd_aggr_2010_2019 = data_2010_2019.filter(lambda x: "AGGRAVATED ASSAULT" in x[9])
cd_aggr_2020_ = data_2020.filter(lambda x: "AGGRAVATED ASSAULT" in x[9])

#For each category, adjust the age limits, then combine the data from both files and finally sort them on descending order by Victim Age
cd_kids_2010_2019 = cd_aggr_2010_2019.filter(lambda x: int(x[11]) < 18 and int(x[11]) >= 0)
cd_kids_2020_ = cd_aggr_2020_.filter(lambda x: int(x[11]) < 18 and int(x[11]) >= 0)
cd_kids = cd_kids_2010_2019.union(cd_kids_2020_)
sorted_cd_kids = cd_kids.sortBy(lambda x: x[11], ascending=False)

cd_young_adlt_2010_2019 = cd_aggr_2010_2019.filter(lambda x: int(x[11]) >= 18 and int(x[11]) < 25)
cd_young_adlt_2020_ = cd_aggr_2020_.filter(lambda x: int(x[11]) >= 18 and int(x[11]) < 25)
cd_young_adlt = cd_young_adlt_2010_2019.union(cd_young_adlt_2020_)
sorted_cd_young_adlt = cd_young_adlt.sortBy(lambda x: x[11], ascending=False)

cd_adlt_2010_2019 = cd_aggr_2010_2019.filter(lambda x: int(x[11]) >= 25 and int(x[11]) <= 64)
cd_adlt_2020_ = cd_aggr_2020_.filter(lambda x: int(x[11]) >= 25 and int(x[11]) <= 64)
cd_adlt = cd_adlt_2010_2019.union(cd_adlt_2020_)
sorted_cd_adlt = cd_adlt.sortBy(lambda x: x[11], ascending=False)

cd_eldery_2010_2019 = cd_aggr_2010_2019.filter(lambda x: int(x[11]) > 64)
cd_eldery_2020_ = cd_aggr_2020_.filter(lambda x: int(x[11]) > 64)
cd_eldery = cd_eldery_2010_2019.union(cd_eldery_2020_)
sorted_cd_eldery = cd_eldery.sortBy(lambda x: x[11], ascending=False)

end_time = time.time()
ex_time=end_time - start_time

# Take a sample of each RDD
#print("Sample of sorted_cd_kids:")
#print(sorted_cd_kids.take(10))

#print("Sample of sorted_cd_young_adlt:")
#print(sorted_cd_young_adlt.take(10))

#print("Sample of sorted_cd_adlt:")
#print(sorted_cd_adlt.take(10))

#print("Sample of sorted_cd_eldery:")
#print(sorted_cd_eldery.take(10))

print(f"Number of rows in the Kids: {sorted_cd_kids.count()}")
print(f"Number of rows in the Young Adults: {sorted_cd_young_adlt.count()}")
print(f"Number of rows in the Adults: {sorted_cd_adlt.count()}")
print(f"Number of rows in the Eldery: {sorted_cd_eldery.count()}")

print(f"Time taken: {ex_time:.2f} seconds")

The code failed because of a fatal error:
	Session 4220 did not start up in 60 seconds..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [ ]:
#Query 1

# Spark DataFrame code
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
from pyspark.sql.functions import col
import time
spark = SparkSession \
    .builder \
    .appName("DF query 1 execution") \
    .getOrCreate()

crimedata = StructType([
    StructField("DR_NO", StringType(), True),
    StructField("Date Rptd", StringType(), True),
    StructField("DATE OCC", StringType(), True),
    StructField("TIME OCC", StringType(), True),
    StructField("AREA", IntegerType(), True),
    StructField("AREA NAME", StringType(), True),
    StructField("Rpt Dist No", IntegerType(), True),
    StructField("Part 1-2", IntegerType(), True),
    StructField("Crm Cd", IntegerType(), True),
    StructField("Crm Cd Desc", StringType(), True),
    StructField("Mocodes", StringType(), True),
    StructField("Vict Age", IntegerType(), True),
    StructField("Vict Sex", StringType(), True),
    StructField("Vict Descent", StringType(), True),
    StructField("Premis Cd", IntegerType(), True),
    StructField("Premis Desc", StringType(), True),
    StructField("Weapon Used Cd", IntegerType(), True),
    StructField("Weapon Desc", StringType(), True),
    StructField("Status", StringType(), True),
    StructField("Status Desc", StringType(), True),
    StructField("Crm Cd 1", IntegerType(), True),
    StructField("Crm Cd 2", IntegerType(), True),
    StructField("Crm Cd 3", IntegerType(), True),
    StructField("Crm Cd 4", IntegerType(), True),
    StructField("LOCATION", StringType(), True),
    StructField("Cross Street", StringType(), True),
    StructField("LAT", FloatType(), True),
    StructField("LON", FloatType(), True)
])

schema1 = "Crm_Cd_Desc STRING, Vict_Age INT"

# Starting Time
start_time = time.time()

crimedata_2010_2019 = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=False, schema=crimedata)
crimedata_2020_ = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv", header=False, schema=crimedata)

#Filter to get only the incidents with aggravated assault
crimedata_2010_2019_aggr = crimedata_2010_2019.filter(col("Crm Cd Desc").contains("AGGRAVATED ASSAULT"))
crimedata_2020_aggr = crimedata_2020_.filter(col("Crm Cd Desc").contains("AGGRAVATED ASSAULT"))

#For each category, adjust the age limits, then combine the data from both files and finally sort them on descending order by Victim Age
kids2010_2019 = crimedata_2010_2019_aggr.filter((col("Vict Age") >= 0) & (col("Vict Age") < 18))
kids2020_ = crimedata_2020_aggr.filter((col("Vict Age") >= 0) & (col("Vict Age") < 18))
kids = kids2010_2019.union(kids2020_)
sorted_kids = kids.sort(col("Vict Age"), ascending=False)
sorted_kids = kids.orderBy(col("Vict Age").desc())

young_adult2010_2019 = crimedata_2010_2019_aggr.filter((col("Vict Age") >= 18) & (col("Vict Age") <= 24))
young_adult2020_ = crimedata_2020_aggr.filter((col("Vict Age") >= 18) & (col("Vict Age") <= 24))
young_adults = young_adult2010_2019.union(young_adult2020_)
sorted_young_adults = young_adults.orderBy(col("Vict Age").desc())

adult2010_2019 = crimedata_2010_2019_aggr.filter((col("Vict Age") >= 25) & (col("Vict Age") <= 64))
adult2020_ = crimedata_2020_aggr.filter((col("Vict Age") >= 25) & (col("Vict Age") <= 64))
adults = adult2010_2019.union(adult2020_)
sorted_adults = adults.orderBy(col("Vict Age").desc())

elderly2010_2019 = crimedata_2010_2019_aggr.filter(col("Vict Age") > 64)
elderly2020_ = crimedata_2020_aggr.filter(col("Vict Age") > 64)
elderly = elderly2010_2019.union(elderly2020_)
sorted_elderly = elderly.orderBy(col("Vict Age").desc())

#Show how much time did it take
end_time = time.time()
ex_time = end_time - start_time

#Show results
#sorted_kids.show(10)
#sorted_young_adults.show(10)
#sorted_adults.show(10)
#sorted_elderly.show(10)

print(f"Number of rows in the Kids: {sorted_kids.count()}")
print(f"Number of rows in the Young Adults: {sorted_young_adults.count()}")
print(f"Number of rows in the Adults: {sorted_adults.count()}")
print(f"Number of rows in the Eldery: {sorted_elderly.count()}")
print(f"Time taken: {ex_time:.2f} seconds")

In [1]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "4",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "4",
        "spark.driver.memory": "2g"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4084,application_1732639283265_4024,pyspark,idle,Link,Link,None,
4087,application_1732639283265_4027,pyspark,idle,Link,Link,None,
4093,application_1732639283265_4033,pyspark,idle,Link,Link,None,
4094,application_1732639283265_4034,pyspark,idle,Link,Link,None,
4104,application_1732639283265_4044,pyspark,idle,Link,Link,None,
4105,application_1732639283265_4045,pyspark,idle,Link,Link,None,
4106,application_1732639283265_4046,pyspark,idle,Link,Link,None,
4108,application_1732639283265_4048,pyspark,idle,Link,Link,None,
4111,application_1732639283265_4051,pyspark,idle,Link,Link,None,
4112,application_1732639283265_4052,pyspark,idle,Link,Link,None,


In [6]:
# Spark RDD code
from pyspark.sql import SparkSession
# To log our application's execution time:
import time

sc = SparkSession \
    .builder \
    .appName("RDD query 1 execution") \
    .getOrCreate() \
    .sparkContext

start_time = time.time()
#crime data
cd_2010_2019 = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv") \
                    .map(lambda x: (x.split(",")))
cd_2020_ = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv") \
                    .map(lambda x: (x.split(",")))

header_2010_2019 = cd_2010_2019.first()
header_2020 = cd_2020_.first()

data_2010_2019 = cd_2010_2019.filter(lambda x: x != header_2010_2019)
data_2020 = cd_2020_.filter(lambda x: x != header_2020)

cd_aggr_2010_2019 = data_2010_2019.filter(lambda x: "AGGRAVATED ASSAULT" in x[9])
cd_aggr_2020_ = data_2020.filter(lambda x: "AGGRAVATED ASSAULT" in x[9])

cd_kids_2010_2019 = cd_aggr_2010_2019.filter(lambda x: x[11] < "18" and x[11] >= "0")
cd_kids_2020_ = cd_aggr_2020_.filter(lambda x: x[11] < "18" and x[11] > "0")
cd_kids = cd_kids_2010_2019.union(cd_kids_2020_)
sorted_cd_kids = cd_kids.sortBy(lambda x: x[11], ascending=False)

cd_young_adlt_2010_2019 = cd_aggr_2010_2019.filter(lambda x: int(x[11]) >= 18 and int(x[11]) < 25)
cd_young_adlt_2020_ = cd_aggr_2020_.filter(lambda x: int(x[11]) >= 18 and int(x[11]) < 25)
cd_young_adlt = cd_young_adlt_2010_2019.union(cd_young_adlt_2020_)
sorted_cd_young_adlt = cd_young_adlt.sortBy(lambda x: x[11], ascending=False)

cd_adlt_2010_2019 = cd_aggr_2010_2019.filter(lambda x: x[11] >= "25" and x[11] <= "64")
cd_adlt_2020_ = cd_aggr_2020_.filter(lambda x: x[11] >= "25" and x[11] <= "64")
cd_adlt = cd_adlt_2010_2019.union(cd_adlt_2020_)
sorted_cd_adlt = cd_adlt.sortBy(lambda x: x[11], ascending=False)

cd_eldery_2010_2019 = cd_aggr_2010_2019.filter(lambda x: x[11] > "64")
cd_eldery_2020_ = cd_aggr_2020_.filter(lambda x: x[11] > "64")
cd_eldery = cd_eldery_2010_2019.union(cd_eldery_2020_)
sorted_cd_eldery = cd_eldery.sortBy(lambda x: x[11], ascending=False)

end_time = time.time()
time_ex=end_time - start_time

print("Sample of sorted_cd_young_adlt:")
print(sorted_cd_young_adlt.take(10))

print("Sample of sorted_cd_adlt:")
print(sorted_cd_adlt.take(10))

print("Sample of sorted_cd_eldery:")
print(sorted_cd_eldery.take(10))

# Take a sample of each RDD
print("Sample of sorted_cd_kids:")
print(sorted_cd_kids.take(10))
print(time_ex)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Sample of sorted_cd_young_adlt:
[['121906512', '02/13/2012 12:00:00 AM', '02/12/2012 12:00:00 AM', '1225', '19', 'Mission', '1963', '1', '236', 'INTIMATE PARTNER - AGGRAVATED ASSAULT', '2000 0408 0319 0416', '24', 'F', 'H', '502', '"MULTI-UNIT DWELLING (APARTMENT', ' DUPLEX', ' ETC)"', '400', '"STRONG-ARM (HANDS', ' FIST', ' FEET OR BODILY FORCE)"', 'AA', 'Adult Arrest', '236', '', '', '', '9600    WOODMAN                      AV', '', '34.2448', '-118.4467'], ['121210118', '03/29/2012 12:00:00 AM', '03/29/2012 12:00:00 AM', '0830', '12', '77th Street', '1245', '1', '236', 'INTIMATE PARTNER - AGGRAVATED ASSAULT', '2000 1243 1814 0416 1203', '24', 'M', 'B', '501', 'SINGLE FAMILY DWELLING', '500', 'UNKNOWN WEAPON/OTHER WEAPON', 'AA', 'Adult Arrest', '236', '', '', '', '1100 W  GAGE                         AV', '', '33.9823', '-118.2938'], ['121511054', '05/05/2012 12:00:00 AM', '05/05/2012 12:00:00 AM', '1200', '15', 'N Hollywood', '1583', '1', '236', 'INTIMATE PARTNER - AGGRAVATED ASSAU

In [2]:
# Spark DataFrame code
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
from pyspark.sql.functions import col

spark = SparkSession \
    .builder \
    .appName("DF query 1 execution") \
    .getOrCreate()

employees_schema = StructType([
    StructField("id", IntegerType()),
    StructField("name", StringType()),
    StructField("salary", FloatType()),
    StructField("dep_id", IntegerType()),
])

employees_df = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/spark-example-data/employees.csv", header=False, schema=employees_schema)
#print the schema of the DataFrame:
employees_df.printSchema()

## Alternative way to read csv:
# employees_df = spark.read.format('csv') \
#     .options(header='false') \
#     .schema(employees_schema) \
#     .load("s3://initial-notebook-data-bucket-dblab-905418150721/spark-example-data/employees.csv")

sorted_employees_df = employees_df.sort(col("salary"))
sorted_employees_df.show(5)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1886,application_1732639283265_1847,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: float (nullable = true)
 |-- dep_id: integer (nullable = true)

+---+---------+------+------+
| id|     name|salary|dep_id|
+---+---------+------+------+
|  6|  Jerry L| 550.0|     3|
|  2|   John K|1000.0|     2|
|  7| Marios K|1000.0|     1|
|  5|  Helen K|1050.0|     2|
| 10|Yiannis T|1500.0|     1|
+---+---------+------+------+
only showing top 5 rows

In [ ]:
#QUERY 2 WITH DATAFRAMES

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc, dense_rank
from pyspark.sql.window import Window
import time

sc = SparkSession \
    .builder \
    .appName("Query 2") \
    .getOrCreate()


crime_df_2010_2019 = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, inferSchema=True)
crime_df_2020_present = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv", header=True, inferSchema=True)
# Ενοποίηση των δύο DataFrames

crime_df = crime_df_2010_2019.union(crime_df_2020_present)
crime_df = crime_df.select(
    col("DATE OCC").alias("date_occ"),
    col("AREA NAME").alias("precinct"),
    col("Status Desc").alias("status")
)
crime_df = crime_df.withColumn("year", col("date_occ").substr(7, 4))
crime_closed = crime_df.filter(col("status").isin("Adult Arrest", "Adult Other", "Juv Arrest", "Juv Other"))

crime_stats = crime_closed.groupBy("year", "precinct") \
    .agg(
        count("*").alias("closed_cases")
    )
total_cases = crime_df.groupBy("year", "precinct") \
    .agg(
        count("*").alias("total_cases")
    )
crime_percentage = crime_stats.join(total_cases, ["year", "precinct"]) \
    .withColumn("closed_case_rate", (col("closed_cases") / col("total_cases")) * 100)
window_spec = Window.partitionBy("year").orderBy(desc("closed_case_rate"))
crime_top3 = crime_percentage.withColumn("rank", dense_rank().over(window_spec)) \
    .filter(col("rank") <= 3)

crime_top3_sorted = crime_top3.orderBy("year", "rank")

start_time = time.time()
crime_top3_sorted.select("year", "precinct", "closed_case_rate", "rank").show()
end_time = time.time()
dataframe_api_time = end_time - start_time
print(f"Time taken by DataFrame API: {dataframe_api_time} seconds")

Starting Spark application


The code failed because of a fatal error:
	Session 4220 did not start up in 60 seconds..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [ ]:
#QUERY 2 WITH SQL API

In [3]:
# Register the DataFrames as temporary views
crime_df.createOrReplaceTempView("crime_df")
crime_closed.createOrReplaceTempView("crime_closed")
crime_stats.createOrReplaceTempView("crime_stats")
total_cases.createOrReplaceTempView("total_cases")
crime_percentage.createOrReplaceTempView("crime_percentage")
crime_top3.createOrReplaceTempView("crime_top3")


sql_query = """
    WITH ranked_data AS (
        SELECT
            cp.year,
            cp.precinct,
            cp.closed_case_rate,
            RANK() OVER (PARTITION BY cp.year ORDER BY cp.closed_case_rate DESC) AS rank
        FROM
            crime_percentage cp
    )
    SELECT
        year,
        precinct,
        closed_case_rate,
        rank
    FROM
        ranked_data
    WHERE
        rank <= 3
    ORDER BY
        year,
        rank
"""


# Execute the SQL query

start_time_sql = time.time()
crime_top3_sql = sc.sql(sql_query)
crime_top3_sql.show()
end_time_sql = time.time()

sql_api_time = end_time_sql - start_time_sql
print(f"Time taken with SQL API : {sql_api_time} seconds")




The code failed because of a fatal error:
	Session 4220 did not start up in 60 seconds..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [ ]:
# QUERY 2 USING DATAFRAMES:  CSV VS PARQUET FORMAT

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc, dense_rank
from pyspark.sql.window import Window
import time

# Δημιουργία Spark Session
sc = SparkSession \
    .builder \
    .appName("Query 2 - CSV vs Parquet") \
    .getOrCreate()

crime_df_2010_2019 = sc.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, inferSchema=True)
crime_df_2020_present = sc.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv", header=True, inferSchema=True)

# Ενοποίηση των δεδομένων
crime_df = crime_df_2010_2019.union(crime_df_2020_present)

# Αποθήκευση σε μοναδικό Parquet αρχείο
crime_df.write.mode("overwrite").parquet("s3://groups-bucket-dblab-905418150721/group26/CrimeData.parquet")

print("Dataset saved in Parquet format to S3.")

# Χρονόμετρο για CSV
start_time_csv = time.time()

# Διαβάζουμε τα δεδομένα από CSV
crime_df = sc.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, inferSchema=True) \
    .union(sc.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv", header=True, inferSchema=True))

crime_df = crime_df.select(
    col("DATE OCC").alias("date_occ"),
    col("AREA NAME").alias("precinct"),
    col("Status Desc").alias("status")
)

crime_df = crime_df.withColumn("year", col("date_occ").substr(7, 4))
crime_closed = crime_df.filter(col("status").isin("Adult Arrest", "Adult Other", "Juv Arrest", "Juv Other"))

crime_stats = crime_closed.groupBy("year", "precinct") \
    .agg(count("*").alias("closed_cases"))
total_cases = crime_df.groupBy("year", "precinct") \
    .agg(count("*").alias("total_cases"))

crime_percentage = crime_stats.join(total_cases, ["year", "precinct"]) \
    .withColumn("closed_case_rate", (col("closed_cases") / col("total_cases")) * 100)

window_spec = Window.partitionBy("year").orderBy(desc("closed_case_rate"))
crime_top3 = crime_percentage.withColumn("rank", dense_rank().over(window_spec)) \
    .filter(col("rank") <= 3)

crime_top3_sorted = crime_top3.orderBy("year", "rank")
#crime_top3_sorted.show()

end_time_csv = time.time()
csv_time = end_time_csv - start_time_csv
print(f"Time taken with CSV input: {csv_time} seconds")


# Χρονόμετρο για Parquet
start_time_parquet = time.time()

# Διαβάζουμε τα δεδομένα από Parquet
crime_df = sc.read.parquet("s3://groups-bucket-dblab-905418150721/group26/CrimeData.parquet")

crime_df = crime_df.select(
    col("DATE OCC").alias("date_occ"),
    col("AREA NAME").alias("precinct"),
    col("Status Desc").alias("status")
)

crime_df = crime_df.withColumn("year", col("date_occ").substr(7, 4))
crime_closed = crime_df.filter(col("status").isin("Adult Arrest", "Adult Other", "Juv Arrest", "Juv Other"))

crime_stats = crime_closed.groupBy("year", "precinct") \
    .agg(count("*").alias("closed_cases"))
total_cases = crime_df.groupBy("year", "precinct") \
    .agg(count("*").alias("total_cases"))

crime_percentage = crime_stats.join(total_cases, ["year", "precinct"]) \
    .withColumn("closed_case_rate", (col("closed_cases") / col("total_cases")) * 100)

window_spec = Window.partitionBy("year").orderBy(desc("closed_case_rate"))
crime_top3 = crime_percentage.withColumn("rank", dense_rank().over(window_spec)) \
    .filter(col("rank") <= 3)

crime_top3_sorted = crime_top3.orderBy("year", "rank")
#crime_top3_sorted.show()

end_time_parquet = time.time()
parquet_time = end_time_parquet - start_time_parquet
print(f"Time taken with Parquet input: {parquet_time} seconds")

# -----------------------------------------------
# Βήμα 4: Σύγκριση Χρόνων Εκτέλεσης
# -----------------------------------------------
print(f"CSV execution time: {csv_time} seconds")
print(f"Parquet execution time: {parquet_time} seconds")
print(f"Parquet was faster by: {csv_time - parquet_time} seconds")


In [ ]:
#QUERY 3 USING DATAFRAMES

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, regexp_replace, trim, coalesce, lit,explode, when
from pyspark.sql import functions as F

# Initialize SparkSession
sc = SparkSession \
    .builder \
    .appName("Query 3") \
    .getOrCreate()
############################################# AVERAGE INCOME PER PERSON PER AREA ######################################################


######################################################   DATASET PREPROCESSING ########################################################

##  INCOME DATASET ##

income_df = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv", header=True, inferSchema=True)


income_df_cleaned = income_df.select(
    col("Zip Code").alias("income_zip"),
    regexp_replace(col("Estimated Median Income"), "[$,]", "").cast("int").alias("income"),
    col("Community").alias("area_income")  # Renaming area column to avoid ambiguity
)

#split the records that have multiple areas in the community 
income_split_df = income_df_cleaned.withColumn("area_income", F.explode(split(col("area_income"), r',\s*|\(\s*|\s*\)')))
income_split_df = income_split_df.filter(
    (col("area_income").isNotNull()) & (col("area_income") != "")
)
# Trim whitespace from area names
income_split_df = income_split_df.withColumn("area_income", trim(col("area_income")))


##  POPULATION DATASET  ##
population_df = spark.read.option("multiline", "true").json("s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson")

# Explode the 'features' array to access individual feature objects
exploded_df = population_df.select(explode(col("features")).alias("feature"))

cleaned_population_df = exploded_df.select(
    col("feature.properties.COMM").alias("area"),
    col("feature.properties.POP_2010").alias("population"),
    col("feature.properties.ZCTA10").alias("population_zip"),
    col("feature.properties.HOUSING10").alias("households"),
    col("feature.properties.CITY").alias("city")
)

# Filter out rows with null or empty zip codes
cleaned_population_df = cleaned_population_df.filter(
    (col("city") == "Los Angeles") &
    (col("population_zip").isNotNull()) & (col("population_zip") != "") &
    (col("households").isNotNull()) & (col("households") > 0) &
     (col("population").isNotNull()) & (col("population") > 0)
   
)
cleaned_population_df = cleaned_population_df.withColumn("area", trim(col("area")))
cleaned_population_df = cleaned_population_df.filter(
    (col("area").isNotNull()) & (col("area") != "")
)
#cleaned_population_df.show() #This dataframe contains the areas, population, zip and number of households per block


####################################################### AVERAGE HOUSEHOLD SIZE PER AREA ##############################################

#Now i will calculate the total population and total households for the areas with the same name and same zip code. So the total for every zip code./
aggregated_population_df = cleaned_population_df.groupBy("area", "population_zip").agg(
    F.sum("population").alias("total_population"),
    F.sum("households").alias("total_households")
)

sorted_aggregated_population_df = aggregated_population_df.orderBy("area", "population_zip")
sorted_aggregated_population_df.show()

#Now i will coacluate the average household size per zip
aggregated_population_df = aggregated_population_df.withColumn(
    "avg_household_size_per_zip",
    when(col("total_households") > 0, col("total_population") / col("total_households")).otherwise(None)
)
sorted_aggregated_population_df =  aggregated_population_df.orderBy("area")
#sorted_aggregated_population_df.show()

#Now i will calculate the average household size per area 
average_household_size_per_area = aggregated_population_df.groupBy("area").agg(
    F.sum((col("avg_household_size_per_zip") * col("total_households"))).alias("weighted_sum"),
    F.sum("total_households").alias("total_households_per_area")
)

average_household_size_per_area = average_household_size_per_area.withColumn(
    "avg_household_size_per_area",
    when(col("total_households_per_area") > 0, col("weighted_sum") / col("total_households_per_area")).otherwise(None)
)

################################################### AVERAGE INCOME PER PERSON PER ZIP ################################################

#For the same area, each zip code has its own mean income so I will first calculate the mean income per person for every zip
joined_df = aggregated_population_df.join(
    income_split_df,
    #(aggregated_population_df["area"] == income_split_df["area_income"]) &
    (aggregated_population_df["population_zip"] == income_split_df["income_zip"]),
    how="inner"
)


joined_df = joined_df.select(
    "area", "population_zip", "total_population", "total_households",
    "avg_household_size_per_zip", "income"
).orderBy("area", "population_zip")

cleaned_joined_df = joined_df.filter(
    (col("area").isNotNull()) & (col("area") != "")
)
cleaned_joined_df = cleaned_joined_df.filter(
    (col("population_zip").isNotNull()) & (col("population_zip") != "")
)
#cleaned_joined_df.show()


#Now i will calculate the average income per person per area 
joined_df = cleaned_joined_df.withColumn(
    "total_income_per_zip",
    col("income") * col("total_households")
)

aggregated_income_population_df = joined_df.groupBy("area").agg(
    F.sum("total_income_per_zip").alias("total_income_area"),
    F.sum("total_population").alias("total_population_area")
)

aggregated_income_population_df = aggregated_income_population_df.withColumn(
    "avg_income_per_person_per_area",
    when(col("total_population_area") > 0, col("total_income_area") / col("total_population_area")).otherwise(None)
)




########################################################## CRIME RATE PER PERSON #####################################################

######################################################   DATASET PREPROCESSING ########################################################
import sedona
from sedona.spark import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg

# Create Spark Session
spark = SparkSession.builder \
    .appName("GeoJSON read") \
    .getOrCreate()

# Create Sedona context
sedona = SedonaContext.create(spark)

# Read the GeoJSON file from S3
geojson_path = "s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson"
blocks_df = sedona.read.format("geojson") \
            .option("multiLine", "true").load(geojson_path) \
            .selectExpr("explode(features) as features") \
            .select("features.*")

# Formatting the GeoJSON data
flattened_df = blocks_df.select( \
                [col(f"properties.{col_name}").alias(col_name) for col_name in \
                blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type")

# Print schema for the GeoJSON data
#flattened_df.printSchema()

# Read the Crime data files
crime_df_2010_2019 = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, inferSchema=True)
crime_df_2020_present = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv", header=True, inferSchema=True)
crime_df = crime_df_2010_2019.union(crime_df_2020_present)

# Clean the crime data to extract necessary columns (crime_id, LAT, LON)
df1 = crime_df.select(
    col("DR_NO").alias("crime_id"),  # Crime ID (DR_NO)
    col("LAT").alias("latitude"),  # Latitude
    col("LON").alias("longitude")  # Longitude
)

df1 = df1.withColumn("geom", ST_Point("longitude", "latitude"))


joined_df1 = df1 \
    .join(flattened_df, ST_Within(df1.geom, flattened_df.geometry), "inner")


final_df = joined_df1.select(
    col("COMM").alias("area"),  # Area name (community)
    col("POP_2010").alias("pop10"),  # Population of the block
    col("geometry"),  # Geometry of the block
    col("latitude").alias("latitude"),  # Latitude of the crime location
    col("longitude").alias("longitude"),  # Longitude of the crime location
    col("crime_id")  # Crime ID
)
#final_df.show()

# Calculate the number of crimes per area
crime_count_df = final_df.groupBy("area", "pop10").agg(
    count("crime_id").alias("num_crimes")
)
#This dataframe contain the average crime rate per person per block
crime_rate_df = crime_count_df.withColumn(
    "crime_rate_per_person",
    col("num_crimes") / col("pop10")
)
filtered_df = crime_rate_df.filter(col("area").isNotNull() & (col("area") != ""))

#I will calculate the crime rate per area per person, by finding the average.
average_crime_rate_df = filtered_df.groupBy("area").agg(
    avg("crime_rate_per_person").alias("avg_crime_rate_per_person")
)

############################################# FINAL RESULTS INTO ONE TABLE ###########################################################

final_crime_rate_per_area_per_person = average_crime_rate_df.orderBy("area")


final_result_df = aggregated_income_population_df.join(
    final_crime_rate_per_area_per_person,
    on="area",
    how="inner" 
)

final_result_df = final_result_df.select(
    "area",
    "avg_income_per_person_per_area",
    "avg_crime_rate_per_person"
)

final_result_df.orderBy("area").show()

In [ ]:
joined_df.explain(mode="formatted")

In [ ]:
joined_df1.explain(mode="formatted")

In [ ]:
final_result_df.explain(mode="formatted")

In [ ]:

############################################## TESTING DIFFERENT JOIN STRATEGIES ######################################################
from pyspark.sql import functions as F
from time import time

# Συνάρτηση για μέτρηση απόδοσης join
def measure_join_performance(join_type_hint, df1, df2, join_condition, join_type="inner"):
    """
    Μετράει την απόδοση ενός join με συγκεκριμένο hint στρατηγικής.
    """
    df2_with_hint = df2.hint(join_type_hint)
    
    start_time = time()
    joined_df = df1.join(df2_with_hint, join_condition, how=join_type)
    joined_df.count() 
    execution_time = time() - start_time
    
    print(f"Join type: {join_type_hint}, Execution time: {execution_time:.2f} seconds")
    return execution_time


strategies = ["broadcast", "merge", "shuffle_hash", "shuffle_replicate_nl"]


results = {}

#1st join
join_condition_1 = (
    (aggregated_population_df["area"] == income_split_df["area_income"]) &
    (aggregated_population_df["population_zip"] == income_split_df["income_zip"])
)

results["join_1"] = {}
print("Testing strategies for Join 1:")
for strategy in strategies:
    execution_time = measure_join_performance(
        join_type_hint=strategy,
        df1=aggregated_population_df,
        df2=income_split_df,
        join_condition=join_condition_1,
        join_type="left"
    )
    results["join_1"][strategy] = execution_time

#2nd join
join_condition_2 = ST_Within(df1.geom, flattened_df.geometry)

results["join_2"] = {}
print("\nTesting strategies for Join 2:")
for strategy in strategies:
    execution_time = measure_join_performance(
        join_type_hint=strategy,
        df1=df1,
        df2=flattened_df,
        join_condition=join_condition_2,
        join_type="inner"
    )
    results["join_2"][strategy] = execution_time

#3rd join
join_condition_3 = "area"

results["join_3"] = {}
print("\nTesting strategies for Join 3:")
for strategy in strategies:
    execution_time = measure_join_performance(
        join_type_hint=strategy,
        df1=aggregated_income_population_df,
        df2=final_crime_rate_per_area_per_person,
        join_condition=join_condition_3,
        join_type="inner"
    )
    results["join_3"][strategy] = execution_time

#results
print("\nPerformance Results:")
for join_name, join_results in results.items():
    print(f"\n{join_name}:")
    sorted_results = sorted(join_results.items(), key=lambda x: x[1])
    for strategy, time_taken in sorted_results:
        print(f"Strategy: {strategy}, Time: {time_taken:.2f} seconds")
        
# Join 1: Aggregated population and income split data
join_1_broadcast = aggregated_population_df \
    .hint("broadcast") \
    .join(income_split_df, 
          (aggregated_population_df["area"] == income_split_df["area_income"]) & 
          (aggregated_population_df["population_zip"] == income_split_df["income_zip"]), 
          how="left")

join_1_merge = aggregated_population_df \
    .hint("merge") \
    .join(income_split_df, 
          (aggregated_population_df["area"] == income_split_df["area_income"]) & 
          (aggregated_population_df["population_zip"] == income_split_df["income_zip"]), 
          how="left")

join_1_shuffle_hash = aggregated_population_df \
    .hint("shuffle_hash") \
    .join(income_split_df, 
          (aggregated_population_df["area"] == income_split_df["area_income"]) & 
          (aggregated_population_df["population_zip"] == income_split_df["income_zip"]), 
          how="left")

join_1_shuffle_replicate_nl = aggregated_population_df \
    .hint("shuffle_replicate_nl") \
    .join(income_split_df, 
          (aggregated_population_df["area"] == income_split_df["area_income"]) & 
          (aggregated_population_df["population_zip"] == income_split_df["income_zip"]), 
          how="left")



In [ ]:
#QUERY 4

In [177]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "1",
        "spark.driver.memory": "2g"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1856,application_1732639283265_1817,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1819,application_1732639283265_1780,pyspark,idle,Link,Link,None,
1834,application_1732639283265_1795,pyspark,idle,Link,Link,None,
1835,application_1732639283265_1796,pyspark,idle,Link,Link,None,
1836,application_1732639283265_1797,pyspark,idle,Link,Link,None,
1841,application_1732639283265_1802,pyspark,idle,Link,Link,None,
1843,application_1732639283265_1804,pyspark,idle,Link,Link,None,
1848,application_1732639283265_1809,pyspark,idle,Link,Link,None,
1849,application_1732639283265_1810,pyspark,idle,Link,Link,None,
1854,application_1732639283265_1815,pyspark,idle,Link,Link,None,
1856,application_1732639283265_1817,pyspark,idle,Link,Link,None,✔


In [181]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.memory": "4g",
        "spark.executor.cores": "2",
        "spark.driver.memory": "2g"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1857,application_1732639283265_1818,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1819,application_1732639283265_1780,pyspark,idle,Link,Link,None,
1834,application_1732639283265_1795,pyspark,idle,Link,Link,None,
1835,application_1732639283265_1796,pyspark,idle,Link,Link,None,
1836,application_1732639283265_1797,pyspark,idle,Link,Link,None,
1841,application_1732639283265_1802,pyspark,idle,Link,Link,None,
1843,application_1732639283265_1804,pyspark,idle,Link,Link,None,
1848,application_1732639283265_1809,pyspark,idle,Link,Link,None,
1849,application_1732639283265_1810,pyspark,idle,Link,Link,None,
1854,application_1732639283265_1815,pyspark,idle,Link,Link,None,
1857,application_1732639283265_1818,pyspark,idle,Link,Link,None,✔


In [184]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.memory": "8g",
        "spark.executor.cores": "4",
        "spark.driver.memory": "2g"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1858,application_1732639283265_1819,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1819,application_1732639283265_1780,pyspark,idle,Link,Link,None,
1834,application_1732639283265_1795,pyspark,idle,Link,Link,None,
1835,application_1732639283265_1796,pyspark,idle,Link,Link,None,
1836,application_1732639283265_1797,pyspark,idle,Link,Link,None,
1841,application_1732639283265_1802,pyspark,idle,Link,Link,None,
1843,application_1732639283265_1804,pyspark,idle,Link,Link,None,
1848,application_1732639283265_1809,pyspark,idle,Link,Link,None,
1849,application_1732639283265_1810,pyspark,idle,Link,Link,None,
1854,application_1732639283265_1815,pyspark,idle,Link,Link,None,
1858,application_1732639283265_1819,pyspark,idle,Link,Link,None,✔


In [186]:
# Query 4 with DataFrame
import time
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
from pyspark.sql.functions import col, to_timestamp, year, count

spark = SparkSession \
    .builder \
    .appName("DF query 4 execution") \
    .getOrCreate()

start_time = time.time()
crimedata = StructType([
    StructField("DR_NO", StringType(), True),
    StructField("Date Rptd", StringType(), True),
    StructField("DATE OCC", StringType(), True),
    StructField("TIME OCC", StringType(), True),
    StructField("AREA", IntegerType(), True),
    StructField("AREA NAME", StringType(), True),
    StructField("Rpt Dist No", IntegerType(), True),
    StructField("Part 1-2", IntegerType(), True),
    StructField("Crm Cd", IntegerType(), True),
    StructField("Crm Cd Desc", StringType(), True),
    StructField("Mocodes", StringType(), True),
    StructField("Vict Age", IntegerType(), True),
    StructField("Vict Sex", StringType(), True),
    StructField("Vict Descent", StringType(), True),
    StructField("Premis Cd", IntegerType(), True),
    StructField("Premis Desc", StringType(), True),
    StructField("Weapon Used Cd", IntegerType(), True),
    StructField("Weapon Desc", StringType(), True),
    StructField("Status", StringType(), True),
    StructField("Status Desc", StringType(), True),
    StructField("Crm Cd 1", IntegerType(), True),
    StructField("Crm Cd 2", IntegerType(), True),
    StructField("Crm Cd 3", IntegerType(), True),
    StructField("Crm Cd 4", IntegerType(), True),
    StructField("LOCATION", StringType(), True),
    StructField("Cross Street", StringType(), True),
    StructField("LAT", FloatType(), True),
    StructField("LON", FloatType(), True)
])

REdata = StructType([
    StructField("Vict_Desc", StringType(), True),
    StructField("Vict_Desc_Full", StringType(), True),
])

crimedata_2010_2019 = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=False, schema=crimedata)
RE_codes = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/RE_codes.csv", header=False, schema=REdata)

crime_df_2015 = crimedata_2010_2019 \
    .withColumn("DATE OCC", to_timestamp(col("DATE OCC"), "MM/dd/yyyy hh:mm:ss a")) \
    .withColumn("DATE OCC", year(col("DATE OCC"))) \
    .filter(col("DATE OCC") == 2015) \
    .filter(col("Vict Descent").isNotNull())

crime_df_2015 = crime_df_2015.select(
    col("DATE OCC").alias("date_occ"),
    col("AREA NAME").alias("precinct"),
    col("Vict Descent").alias("vict_descent"),
    col("DR_NO").alias("crime_id"),
    col("LAT").alias("latitude"),
    col("LON").alias("longitude")
).withColumn("geom", ST_Point("longitude", "latitude"))

# Top 3 areas by income excluding NULL and 0.0 values
top_3_areas_by_income = aggregated_income_population_df.filter(
    col("avg_income_per_person_per_area").isNotNull() & (col("avg_income_per_person_per_area") > 0.0)).orderBy(
    col("avg_income_per_person_per_area").desc()).limit(3)
#top_3_areas_by_income.select("area", "avg_income_per_person_per_area").show()

# Bottom 3 areas by income excluding NULL and 0.0 values
bottom_3_areas_by_income = aggregated_income_population_df.filter(
    col("avg_income_per_person_per_area").isNotNull() & (col("avg_income_per_person_per_area") > 0.0)).orderBy(
    col("avg_income_per_person_per_area").asc()).limit(3)
#bottom_3_areas_by_income.select("area", "avg_income_per_person_per_area").show()

joined_df_4 = crime_df_2015.join(flattened_df, ST_Within(crime_df_2015.geom, flattened_df.geometry), "inner")

# Get a list of top and bottom areas
top_areas = [row['area'] for row in top_3_areas_by_income.collect()]
bottom_areas = [row['area'] for row in bottom_3_areas_by_income.collect()]

final_df_4 = joined_df_4.select(
    col("COMM").alias("area"),
    col("precinct").alias("precinct"),  # Area name (community)
    col("date_occ").alias("date_occ"),  # Population of the block
    col("geometry"),  # Geometry of the block
    col("latitude").alias("latitude"),  # Latitude of the crime location
    col("longitude").alias("longitude"),  # Longitude of the crime location
    col("vict_descent")  # Crime ID
)

crime_top_areas = final_df_4.filter(col("area").isin(top_areas))
# crime_top_areas.show()
crime_bottom_areas = final_df_4.filter(col("area").isin(bottom_areas))
# crime_bottom_areas.show()

# Join with RE_codes to get the full description
crime_top_areas_with_re = crime_top_areas.join(RE_codes, crime_top_areas["vict_descent"] == RE_codes["Vict_Desc"], "left")
crime_bottom_areas_with_re = crime_bottom_areas.join(RE_codes, crime_bottom_areas["vict_descent"] == RE_codes["Vict_Desc"], "left")
#crime_top_areas_with_re.show(5)

# Group by Victim Descent and count the number of victims
top_areas_victim_count = crime_top_areas_with_re.groupBy("Vict_Desc_Full").agg(count("*").alias("victim_count")).orderBy(col("victim_count").desc())
bottom_areas_victim_count = crime_bottom_areas_with_re.groupBy("Vict_Desc_Full").agg(count("*").alias("victim_count")).orderBy(col("victim_count").desc())

# Show the results
print("Top 3 Areas by Income - Victim Descent Count:")
top_areas_victim_count.show()

print("Bottom 3 Areas by Income - Victim Descent Count:")
bottom_areas_victim_count.show()

end_time = time.time()
ex_time = end_time - start_time
print(f"Time taken: {ex_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Top 3 Areas by Income - Victim Descent Count:
+--------------------+------------+
|      Vict_Desc_Full|victim_count|
+--------------------+------------+
|               White|         159|
|               Other|          26|
|Hispanic/Latin/Me...|          23|
|               Black|          15|
|         Other Asian|           8|
|             Unknown|           5|
+--------------------+------------+

Bottom 3 Areas by Income - Victim Descent Count:
+--------------------+------------+
|      Vict_Desc_Full|victim_count|
+--------------------+------------+
|Hispanic/Latin/Me...|         712|
|               Black|         160|
|               White|         126|
|             Unknown|          94|
|               Other|          93|
|         Other Asian|          34|
|              Korean|          18|
|            Filipino|           3|
+--------------------+------------+

Time taken: 51.89 seconds

In [169]:
final_df_4.select("area").distinct() \
    .filter(col("area").startswith("M")) \
    .orderBy("area") \
    .show()

# Show distinct areas from cleaned_population_df that start with 'M', sorted by area
cleaned_population_df.select("area").distinct() \
    .filter(col("area").startswith("M")) \
    .orderBy("area") \
    .show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|             area|
+-----------------+
|Manchester Square|
|Mandeville Canyon|
|        Mar Vista|
| Marina Peninsula|
|   Marina del Rey|
|          Melrose|
|         Mid-city|
|     Miracle Mile|
|    Mission Hills|
|   Mt. Washington|
+-----------------+

+-----------------+
|             area|
+-----------------+
|           Malibu|
|Manchester Square|
|Mandeville Canyon|
|  Manhattan Beach|
|        Mar Vista|
| Marina Peninsula|
|   Marina del Rey|
|          Maywood|
|          Melrose|
|         Mid-city|
|     Miracle Mile|
|    Mission Hills|
|         Monrovia|
|       Montebello|
|    Monterey Park|
|   Mt. Washington|
+-----------------+

In [97]:
#QUERY 5

An error was encountered:
Invalid status code '404' from http://ec2-35-159-120-182.eu-central-1.compute.amazonaws.com:8998/sessions/1471 with error payload: {"msg":"Session '1471' not found."}


In [101]:
import time
from pyspark.sql.functions import format_number
from pyspark.sql.window import Window

start_time = time.time()
police_stations = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/LA_Police_Stations.csv", header=True, inferSchema=True)
police_stations = police_stations.withColumn('X', format_number('X', 10))  # Adjust the number of decimals as needed
police_stations = police_stations.withColumn('Y', format_number('Y', 10))
police_stations_df = police_stations.withColumn("geom", ST_Point(col("X"), col("Y")))
crime_df = df1.filter(
    (F.col("longitude") != 0) & (F.col("latitude") != 0)
)
crime_df = crime_df.withColumnRenamed("geom", "crime_geom")
police_stations_df = police_stations_df.withColumnRenamed("geom", "police_geom")

joined_df = crime_df.crossJoin(police_stations_df)
joined_df = joined_df.withColumn("distance", ST_DistanceSphere(col("crime_geom"), col("police_geom"))/1000)
joined_df = joined_df.select(
     col("crime_id").alias("crime_id"),
    col("distance").alias("distance"),
    col("FID").alias("police id"),
    col("DIVISION").alias("division")
)

joined_df = joined_df.orderBy("crime_id")
window_spec = Window.partitionBy("crime_id").orderBy("distance")
df_with_rank = joined_df.withColumn("rank", F.row_number().over(window_spec))
filtered_df = df_with_rank.filter(F.col("rank") == 1)
filtered_df = filtered_df.orderBy("police id")
crime_count_and_avg_distance = filtered_df.groupBy("police id", "division").agg(
    F.count("crime_id").alias("crime_count"),
    F.avg("distance").alias("avg_distance")
)
crime_count_and_avg_distance_sorted = crime_count_and_avg_distance.orderBy(F.col("crime_count").desc())

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution Time: {execution_time} seconds")

crime_count_and_avg_distance_sorted.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-------------+---+----------------+--------------------+----+
|              X|            Y|FID|        DIVISION|            LOCATION|PREC|
+---------------+-------------+---+----------------+--------------------+----+
|-118.2892415530|33.7576608970|  1|          HARBOR|2175 JOHN S. GIBS...|   5|
|-118.2753942060|33.9386273800|  2|       SOUTHEAST|    145 W. 108TH ST.|  18|
|-118.2776696550|33.9703073800|  3|     77TH STREET|    7600 S. BROADWAY|  12|
|-118.4198415760|33.9916553210|  4|         PACIFIC|  12312 CULVER BLVD.|  14|
|-118.3051415630|34.0105753400|  5|       SOUTHWEST|1546 MARTIN LUTHE...|   3|
|-118.2561188910|34.0123559050|  6|          NEWTON|3400 S. CENTRAL AVE.|  13|
|-118.2472941230|34.0440195000|  7|         CENTRAL|      251 E. 6TH ST.|   1|
|-118.4507795410|34.0437774120|  8|WEST LOS ANGELES|    1663 BUTLER AVE.|   8|
|-118.2130679560|34.0450087690|  9|      HOLLENBECK|     2111 E. 1ST ST.|   4|
|-118.3428295250|34.0467476820| 10|        WILSHIRE|

In [ ]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.memory": "8g",
        "spark.executor.cores": "4",
        "spark.driver.memory": "2g"
    }
}

In [ ]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "4",
        "spark.executor.memory": "4g",
        "spark.executor.cores": "2",
        "spark.driver.memory": "2g"
    }
}

In [ ]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "8",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "1",
        "spark.driver.memory": "2g"
    }
}